## Dependencies

In [ ]:
!pip install --pre -I "weaviate-client==4.*"

In [ ]:
!pip install weaviate-demo-datasets

## Connect to a local instance

In [7]:
import weaviate, os, json
import weaviate.classes as wvc

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']  # Replace with your inference API key
    }
)

## Create the base collections

In [8]:
if (client.collections.exists("JeopardyCategory")):
    client.collections.delete("JeopardyCategory")

if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyCategory",
    description="A Jeopardy! category",
    properties=[
        wvc.Property(name="title", data_type=wvc.DataType.TEXT),
    ]
)

# CrossRefDefinition
client.collections.create(
    name="JeopardyQuestion",
    description="A Jeopardy! question",
    properties=[
        wvc.Property(name="question", data_type=wvc.DataType.TEXT),
        wvc.Property(name="answer", data_type=wvc.DataType.TEXT),
        # highlight-start
        wvc.ReferenceProperty(
            name="hasCategory",
            target_collection="JeopardyCategory"
        )
        # highlight-end
    ]
)

client.collections.list_all()

{'JeopardyCategory': _CollectionConfigSimple(name='JeopardyCategory', description='A Jeopardy! category', properties=[_Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='title', tokenization=<Tokenization.WORD: 'word'>)], vectorizer=<Vectorizer.NONE: 'none'>),
 'JeopardyQuestion': _CollectionConfigSimple(name='JeopardyQuestion', description='A Jeopardy! question', properties=[_Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='question', tokenization=<Tokenization.WORD: 'word'>), _Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='answer', tokenization=<Tokenization.WORD: 'word'>), _Property(data_type=_ReferenceDataType(target_collection='JeopardyCategory'), description=None, index_filterable=True, index_searchable=False, name='hasCategory', tokenization=None)], vectorizer=<Vectorizer.NONE: 'none'>)}

## Import data

In [9]:
import weaviate_datasets
dataset = weaviate_datasets.JeopardyQuestions1k()  # instantiate dataset

old_client = weaviate.Client("http://localhost:8080")
dataset = weaviate_datasets.JeopardyQuestions1k()  # instantiate dataset
dataset.upload_objects(old_client, 100)

1000it [00:05, 174.65it/s]


True

## Test example

In [10]:
jeopardy = client.collections.get("JeopardyQuestion")
result = jeopardy.query.fetch_objects(limit=10)

# print result properties 
for o in result.objects:
    print(o.properties)

{'points': 500.0, 'answer': 'George Rogers Clark', 'air_date': '1997-07-08T00:00:00Z', 'round': 'Jeopardy!', 'question': 'This soldier & frontiersman won important victories over the British in the Northwest Territory'}
{'points': 400.0, 'air_date': '1993-07-09T00:00:00Z', 'answer': 'Anwar Sadat', 'question': 'On his 1977 visit to Jerusalem, this Egyptian president prayed at the al-Aqsa mosque', 'round': 'Double Jeopardy!'}
{'points': 400.0, 'answer': 'San Francisco', 'air_date': '2008-11-10T00:00:00Z', 'round': 'Double Jeopardy!', 'question': "This city's Cable Car Museum has one from fhe first company, the Clay Street Hill Railroad"}
{'points': 100.0, 'air_date': '1997-05-30T00:00:00Z', 'answer': 'Theodore Roosevelt', 'round': 'Jeopardy!', 'question': 'In 1912 this "Bull Moose" spoke for about 50 minutes with a would-be-assassin\'s bullet lodged in his chest'}
{'points': 200.0, 'air_date': '1996-09-04T00:00:00Z', 'answer': 'Baton Rouge', 'round': 'Jeopardy!', 'question': 'Louisiana S